Seasonal Causal network for raw data

- Jarque-Bera residual test
- building networks
- visualizing network nodes (ingrid, outgrid)
- analysing edges attributes ( number of connections, strenght, direction, spatial lenght)
- network analysis: Clustering Coefficient and Average Shortest Path Length Across Year
- PDF and Decumulative distribution of GC Statistics

In [ ]:
# residuals and networks
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from scipy.stats import jarque_bera, norm
import numpy as np

# Load data
gauges_data = pd.read_csv('75gauges.csv')
coordinates_data = pd.read_csv('merged_COO.csv')

# Merge data on STATION
merged_data_filtered = pd.merge(gauges_data, coordinates_data, on='STATION')

# Convert 'DATETIME' to datetime
merged_data_filtered['DATETIME'] = pd.to_datetime(merged_data_filtered['DATETIME'])

# Define the station IDs you want to include in the network
stations_of_interest = [779, 750, 683, 695, 718, 756, 706, 764, 729]

# Filter merged_data to include only stations of interest
merged_data = merged_data_filtered[merged_data_filtered['STATION'].isin(stations_of_interest)]


# Function to perform Granger causality test and manual residual computation
def granger_test_manual(dataframe, column1, column2, max_lag=1):
    try:
        # Create lagged version of the independent variable (column1)
        X = dataframe[[column1]].shift(max_lag).dropna()
        y = dataframe[column2].iloc[max_lag:]  # Dependent variable adjusted for shift
        X = X[:len(y)]  # Align lengths of X and y

        # Add constant to the model
        X = add_constant(X)

        # Fit OLS regression model
        model = OLS(y, X).fit()

        # Calculate residuals
        residuals = model.resid

        # Perform Jarque-Bera test for normality on residuals
        jb_stat, jb_p_value = jarque_bera(residuals)
        normality = "Normal" if jb_p_value > 0.05 else "Not normal"

        # Plot residuals for normal and non-normal cases
        plot_residuals(residuals, normality, column1, column2)

        # Return F-statistic
        f_statistic = model.fvalue
        return f_statistic if model.f_pvalue < 0.01 else 0
    except Exception as e:
        print(f"Error in granger_test_manual: {e}")
        return 0  # Return 0 in case of an error

# Function to plot residuals
def plot_residuals(residuals, normality, column1, column2):
    plt.figure(figsize=(8, 6))

    # Plot histogram of residuals
    plt.hist(residuals, bins=20, density=True, alpha=0.6, color='g', label="Residuals")

    # Fit a normal distribution to the residuals
    mu, std = norm.fit(residuals)

    # Plot the normal distribution
    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, 100)
    p = norm.pdf(x, mu, std)
    plt.plot(x, p, 'k', linewidth=2, label="Normal fit")

    # Title and labels
    plt.title(f'Residuals Distribution: {column1} -> {column2} ({normality})')
    plt.xlabel('Residuals')
    plt.ylabel('Density')
    plt.legend()

    # Show plot
    plt.show()

# Find global maxima for in-links and F-statistics across all years
global_max_links = 0
global_max_f_statistic = 0

for year in range(merged_data['DATETIME'].dt.year.min(), merged_data['DATETIME'].dt.year.max() + 1):
    data_year = merged_data[(merged_data['DATETIME'].dt.year == year) & (merged_data['DATETIME'].dt.month.isin([9, 10, 11]))]  # Filter for March, April, May
    pivot_data = data_year.pivot(index='DATETIME', columns='STATION', values='VALUE').fillna(0)

    # Initialize temporary dictionaries for this year
    temp_in_links_count = {station: 0 for station in pivot_data.columns}
    temp_f_statistics_sum = {station: 0 for station in pivot_data.columns}

    for station1 in pivot_data.columns:
        for station2 in pivot_data.columns:
            if station1 != station2:
                f_statistic = granger_test_manual(pivot_data, station1, station2)
                if f_statistic > 0:
                    temp_in_links_count[station2] += 1
                    temp_f_statistics_sum[station2] += f_statistic

    # Update global maxima if necessary
    year_max_links = max(temp_in_links_count.values())
    year_max_f_statistic = max(temp_f_statistics_sum.values())
    global_max_links = max(global_max_links, year_max_links)
    global_max_f_statistic = max(global_max_f_statistic, year_max_f_statistic)

# Loop over each year for network creation and visualization
for year in range(merged_data['DATETIME'].dt.year.min(), merged_data['DATETIME'].dt.year.max() + 1):
    data_year = merged_data[(merged_data['DATETIME'].dt.year == year) & (merged_data['DATETIME'].dt.month.isin([9, 10, 11]))]  # Filter for March, April, May
    pivot_data = data_year.pivot(index='DATETIME', columns='STATION', values='VALUE').fillna(0)

    # Initialize dictionaries for in-links and sum of F-statistics for incoming links
    in_links_count = {station: 0 for station in pivot_data.columns}
    incoming_f_statistics_sum = {station: 0 for station in pivot_data.columns}

    # Create network
    G = nx.Graph()
    G.add_nodes_from(pivot_data.columns)

    # Add edges and count in-links and F-statistics for incoming links
    for station1 in pivot_data.columns:
        for station2 in pivot_data.columns:
            if station1 != station2:
                f_statistic = granger_test_manual(pivot_data, station1, station2)
                if f_statistic > 0:
                    G.add_edge(station1, station2)
                    in_links_count[station2] += 1
                    incoming_f_statistics_sum[station2] += f_statistic

    # Normalize node sizes based on the sum of incoming F-statistics
    scaling_factor = 150
    node_sizes = [incoming_f_statistics_sum[station] * scaling_factor / global_max_f_statistic for station in G.nodes()]

    # Normalize and set node colors based on in-links count
    norm = Normalize(vmin=0, vmax=global_max_links)
    cmap = plt.cm.coolwarm
    color_map = ScalarMappable(norm=norm, cmap=cmap)
    node_colors = [color_map.to_rgba(in_links_count[station]) for station in G.nodes()]  # Visualization with scaled node sizes and color gradient
    pos = {station: (coordinates_data[coordinates_data['STATION'] == station]['EST'].iloc[0],
                     coordinates_data[coordinates_data['STATION'] == station]['NORD'].iloc[0])
           for station in G.nodes()}

    nx.draw(G, pos, with_labels=True, node_size=node_sizes, node_color=node_colors, node_shape="o", edge_color='none', font_size=7)
    plt.title(f"Network of Gauges ({year}) with Node Sizes and Colors Based on Incoming Links (March-May)")
    #plt.savefig(f"4.9gauges_innetwork_{year}.jpg", dpi=300)  # Save as JPG
    plt.close()  # Close the plot


In [ ]:
#ingrid
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import grangercausalitytests
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from matplotlib.patches import Circle
import numpy as np

# Load data
gauges_data = pd.read_csv('75gauges.csv')
coordinates_data = pd.read_csv('merged_COO.csv')

# Merge data on STATION
merged_data = pd.merge(gauges_data, coordinates_data, on='STATION')


# Convert 'DATETIME' to datetime
merged_data_filtered['DATETIME'] = pd.to_datetime(merged_data_filtered['DATETIME'])

# Define the station IDs you want to include in the network
stations_of_interest = [779, 750, 683, 695, 718,756, 706, 764, 729]

# Filter merged_data to include only stations of interest
merged_data = merged_data_filtered[merged_data_filtered['STATION'].isin(stations_of_interest)]


station_year_data = {}

# Granger causality test function
def granger_test(dataframe, column1, column2, lag=1):
    try:
        gc_test = grangercausalitytests(dataframe[[column1, column2]], maxlag=lag)
        f_statistic = gc_test[lag][0]['ssr_ftest'][0]  # F-statistic
        p_value = gc_test[lag][0]['ssr_ftest'][1]  # p-value
        return f_statistic if p_value < 0.0001 else 0
    except:
        return 0

# Loop over each year for analysis
for year in range(merged_data['DATETIME'].dt.year.min(), merged_data['DATETIME'].dt.year.max() + 1):
    # Filter data for months 3, 4, and 5
    #data_year = merged_data[(merged_data['DATETIME'].dt.year == year) & 
                            #(merged_data['DATETIME'].dt.month.isin([6, 7, 8]))]
    
    data_year = merged_data[(merged_data['DATETIME'].dt.year == year)]
    pivot_data = data_year.pivot(index='DATETIME', columns='STATION', values='VALUE').fillna(0)

    temp_in_links_count = {station: 0 for station in pivot_data.columns}
    temp_f_statistics_sum = {station: 0 for station in pivot_data.columns}

    # Analysis for each station pair
    for station1 in pivot_data.columns:
        for station2 in pivot_data.columns:
            if station1 != station2:
                f_statistic = granger_test(pivot_data, station1, station2, lag=1)  # Change lag value here
                if f_statistic > 0:
                    temp_in_links_count[station2] += 1
                    temp_f_statistics_sum[station2] += f_statistic
                    station_year_data[(station2, year)] = (temp_f_statistics_sum[station2], temp_in_links_count[station2])

    # Update global maxima
    year_max_links = max(temp_in_links_count.values())
    year_max_f_statistic = max(temp_f_statistics_sum.values())
    global_max_links = max(global_max_links, year_max_links)
    global_max_f_statistic = max(global_max_f_statistic, year_max_f_statistic)




# Define the station order dictionary
station_order = {
    779: "TP",
    750: "PA",
    683: "AG",
    695: "CL",
    718: "EN",
    756: "RG",
    706: "CT",
    764: "SR",
    729: "ME"
}


import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import numpy as np

# Supponiamo che station_year_data, station_order, global_max_f_statistic, e global_max_links siano già definiti

# Crea la visualizzazione della griglia 2D
fig, ax = plt.subplots(figsize=(15, 10))
ax.set_facecolor('white')  # Imposta lo sfondo bianco

# Definisce le scale per la dimensione e il colore
max_node_size = .2  # Massima dimensione del nodo
color_norm = plt.Normalize(0, global_max_links)  # Normalizza il conteggio dei collegamenti
color_map = plt.cm.coolwarm  # Mappa dei colori

# Disegna ogni stazione-anno come un cerchio sulla griglia
for (station, year), (f_stat, links) in station_year_data.items():
    x = year
    y = list(station_order.keys()).index(station)  # Ottiene la posizione della stazione in base all'ordine
    size = (f_stat / global_max_f_statistic) * max_node_size  # Scala la dimensione in base a f_stat
    color = color_map(color_norm(links))  # Ottiene il colore in base al numero di collegamenti

    # Crea un cerchio e aggiungilo al grafico
    circle = Circle((x, y), np.sqrt(size), color=color, alpha=0.6)  # Usa la radice quadrata della dimensione per il raggio
    ax.add_patch(circle)

# Imposta etichette degli assi, tacche e limiti
ax.set_xlabel('Year')
ax.set_ylabel('Station ID')
ax.set_xticks(np.arange(merged_data['DATETIME'].dt.year.min(), merged_data['DATETIME'].dt.year.max() + 1))
ax.set_yticks(np.arange(len(station_order)))
ax.set_yticklabels(station_order.values())
ax.set_xlim(merged_data['DATETIME'].dt.year.min() - 1, merged_data['DATETIME'].dt.year.max() + 1)
ax.set_ylim(-1, len(station_order))

plt.grid(False)
plt.title('Inlinks grid')

# Salva il grafico come file JPG con dpi=300
plt.savefig("0.ingrid_visualization9gauges.jpg", dpi=300)

plt.show()

In [ ]:
# outgrid
import pandas as pd
from statsmodels.tsa.stattools import grangercausalitytests
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import numpy as np

# Load data
gauges_data = pd.read_csv('75gauges.csv')
coordinates_data = pd.read_csv('merged_COO.csv')

# Merge data on STATION
merged_data = pd.merge(gauges_data, coordinates_data, on='STATION')


# Convert 'DATETIME' to datetime
merged_data_filtered['DATETIME'] = pd.to_datetime(merged_data_filtered['DATETIME'])

# Define the station IDs you want to include in the network
stations_of_interest = [779, 750, 683, 695, 718,756, 706, 764, 729]

# Filter merged_data to include only stations of interest
merged_data = merged_data_filtered[merged_data_filtered['STATION'].isin(stations_of_interest)]


# Granger causality test function
def granger_test(dataframe, column1, column2, max_lag=1):
    try:
        gc_test = grangercausalitytests(dataframe[[column1, column2]], maxlag=max_lag)
        f_statistic = gc_test[1][0]['ssr_ftest'][0]  # F-statistic
        p_value = gc_test[1][0]['ssr_ftest'][1]  # p-value
        return f_statistic if p_value < 0.0001 else 0
    except:
        return 0

# Initialize variables to find global maxima
global_max_links = 0
global_max_f_statistic = 0
station_year_data = {}

# Loop over each year for analysis
for year in range(merged_data['DATETIME'].dt.year.min(), merged_data['DATETIME'].dt.year.max() + 1):
    # Filter data for months 6, 7, and 8
    #data_year = merged_data[(merged_data['DATETIME'].dt.year == year) & 
     #                       (merged_data['DATETIME'].dt.month.isin([6, 7, 8]))]
    
    data_year = merged_data[(merged_data['DATETIME'].dt.year == year)]
    pivot_data = data_year.pivot(index='DATETIME', columns='STATION', values='VALUE').fillna(0)

    temp_out_links_count = {station: 0 for station in pivot_data.columns}  # Count of outlinks
    temp_f_statistics_sum = {station: 0 for station in pivot_data.columns}

    # Analysis for each station pair
    for station1 in pivot_data.columns:
        for station2 in pivot_data.columns:
            if station1 != station2:
                f_statistic = granger_test(pivot_data, station1, station2)
                if f_statistic > 0:
                    temp_out_links_count[station1] += 1  # Counting outlinks from station1
                    temp_f_statistics_sum[station1] += f_statistic
                    station_year_data[(station1, year)] = (temp_f_statistics_sum[station1], temp_out_links_count[station1])  # Storing outlink data

    # Update global maxima
    year_max_links = max(temp_out_links_count.values())  # Max outlinks
    year_max_f_statistic = max(temp_f_statistics_sum.values())
    global_max_links = max(global_max_links, year_max_links)
    global_max_f_statistic = max(global_max_f_statistic, year_max_f_statistic)

# Define the station order dictionary
station_order = {
    779: "TP",
    750: "PA",
    683: "AG",
    695: "CL",
    718: "EN",
    756: "RG",
    706: "CT",
    764: "SR",
    729: "ME"
}
# Create the visualization of the 2D grid
fig, ax = plt.subplots(figsize=(15, 10))
ax.set_facecolor('white')  # Set background to white

# Define scales for size and color
max_node_size = .2  # Maximum node size
color_norm = plt.Normalize(0, global_max_links)  # Normalize link count
color_map = plt.cm.coolwarm  # Color map

# Draw each station-year as a circle on the grid
for (station, year), (f_stat, links) in station_year_data.items():
    x = year
    y = list(station_order.keys()).index(station)  # Get station position based on order
    size = (f_stat / global_max_f_statistic) * max_node_size  # Scale size based on f_stat
    color = color_map(color_norm(links))  # Get color based on number of links

    # Create a circle and add it to the plot
    circle = Circle((x, y), np.sqrt(size), color=color, alpha=0.6)  # Use square root of size for radius
    ax.add_patch(circle)

# Set axis labels, ticks, and limits
ax.set_xlabel('Year')
ax.set_ylabel('Station ID')
ax.set_xticks(np.arange(merged_data['DATETIME'].dt.year.min(), merged_data['DATETIME'].dt.year.max() + 1))
ax.set_yticks(np.arange(len(station_order)))
ax.set_yticklabels(station_order.values())
ax.set_xlim(merged_data['DATETIME'].dt.year.min() - 1, merged_data['DATETIME'].dt.year.max() + 1)
ax.set_ylim(-1, len(station_order))

plt.grid(False)
plt.title('Outlinks grid')

# Save the plot as a JPG file with dpi=300
plt.savefig("0.outgrid_visualization9gauges.jpg", dpi=300)

plt.show()

In [ ]:
#LINK 2D GRID seasonal

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from math import radians, cos, sin, asin, sqrt
from statsmodels.tsa.stattools import grangercausalitytests

# Haversine formula to calculate the distance between two points on the Earth
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of Earth in kilometers
    return c * r

# Function to perform Granger causality test and return F-statistic
def granger_test(dataframe, column1, column2, max_lag=1):
    try:
        gc_test = grangercausalitytests(dataframe[[column1, column2]], maxlag=max_lag)
        f_statistic = gc_test[1][0]['ssr_ftest'][0]  # F-statistic
        p_value = gc_test[1][0]['ssr_ftest'][1]  # p-value
        return f_statistic if p_value < 0.0001 else 0
    except:
        return 0  # Return 0 in case of an error

# Function to determine the color of a cell based on the link direction and distance
def link_color(lon1, lat1, lon2, lat2, max_distance):
    distance = haversine(lon1, lat1, lon2, lat2)
    normalized_distance = distance / max_distance  # Normalize distance to [0, 1]
    
    if lon2 > lon1:
        # Eastward link, use red colormap
        return mcolors.to_hex(plt.cm.Reds(normalized_distance))
    else:
        # Westward link, use blue colormap
        return mcolors.to_hex(plt.cm.Blues(normalized_distance))

# Load data
gauges_data = pd.read_csv('75gauges.csv')
coordinates_data = pd.read_csv('merged_COO.csv')

# Merge data on STATION
merged_data = pd.merge(gauges_data, coordinates_data, on='STATION')

# Convert 'DATETIME' to datetime
merged_data['DATETIME'] = pd.to_datetime(merged_data['DATETIME'])

# Define the station order dictionary
station_order = {
    779: "TP",
    775: "ERICE",
    780: "TRAPANI FULGATORE",
    778: "SALEMI",
    774: "CASTELVETRANO",
    773: "CASTELLAMMARE DEL GOLFO",
    744: "Contessa Entellina",
    751: "PARTINICO",
    742: "CAMPOREALE",
    749: "MONREALE VIGNA API",
    745: "CORLEONE",
    693: "RIBERA",
    750: "PA",
    686: "BIVONA",
    748: "MISILMERI",
    747: "MEZZOJUSO",
    683: "AG",
    755: "TERMINI IMERESE",
    685: "ARAGONA",
    684: "AGRIGENTO MANDRASCAVA",
    687: "CAMMARATA",
    740: "ALIA",
    689: "CANICATTì",
    700: "MUSSOMELI",
    703: "SCLAFANI BAGNI",
    690: "LICATA",
    746: "LASCARI",
    696: "DELIA",
    754: "POLIZZI GENEROSA",
    753: "PETRALIA SOTTANA",
    695: "CL",
    701: "RIESI",
    743: "CASTELBUONO",
    718: "EN",
    752: "GANGI",
    699: "MAZZARINO",
    736: "PETTINEO",
    731: "MISTRETTA",
    722: "PIAZZA ARMERINA",
    762: "ACATE",
    721: "NICOSIA",
    717: "AIDONE",
    723: "CARONIA BUZZA",
    760: "SANTA CROCE CAMERINA",
    710: "MAZZARRONE",
    716: "CALTAGIRONE",
    757: "COMISO",
    737: "SAN FRATELLO",
    730: "MILITELLO ROSMARINO",
    761: "SCICLI",
    756: "RG",
    725: "CESARò VIGNAZZA",
    711: "MINEO",
    733: "NASO",
    705: "BRONTE",
    712: "PATERNò",
    770: "PALAZZOLO ACREIDE",
    709: "MALETTO",
    765: "FRANCOFONTE",
    766: "LENTINI",
    715: "RANDAZZO",
    758: "ISPICA",
    735: "PATTI",
    713: "PEDARA",
    767: "NOTO",
    706: "CT",
    769: "PACHINO",
    708: "LINGUAGLOSSA",
    734: "NOVARA DI SICILIA",
    764: "SR",
    707: "RIPOSTO",
    739: "TORREGROTTA",
    738: "SAN PIER NICETO",
    727: "FIUMEDINISI",
    729: "ME"
}


# Create a list of station IDs in order
station_list = list(station_order.keys())

# Process data for each year and calculate all F-statistics
all_f_statistics = []
for year in range(merged_data['DATETIME'].dt.year.min(), merged_data['DATETIME'].dt.year.max() + 1):
    data_year = merged_data[(merged_data['DATETIME'].dt.year == year) & (merged_data['DATETIME'].dt.month.isin([9, 10, 11]))]
    pivot_data = data_year.pivot(index='DATETIME', columns='STATION', values='VALUE').fillna(0)

    for station1 in pivot_data.columns:
        for station2 in pivot_data.columns:
            if station1 != station2:
                f_statistic = granger_test(pivot_data, station1, station2)
                all_f_statistics.append(f_statistic)

# Calculate the 99th percentile of F-statistics
f_statistic_99th_percentile = np.percentile(all_f_statistics, 99)

# Initialize a matrix to store the link information for each year
years = range(merged_data['DATETIME'].dt.year.min(), merged_data['DATETIME'].dt.year.max() + 1)
num_stations = len(station_list)
link_matrix = {year: np.zeros((num_stations, num_stations)) for year in years}

# Populate the link matrix
for year in years:
    data_year = merged_data[(merged_data['DATETIME'].dt.year == year) & (merged_data['DATETIME'].dt.month.isin([9, 10,11]))]
    pivot_data = data_year.pivot(index='DATETIME', columns='STATION', values='VALUE').fillna(0)

    for i, station1 in enumerate(station_list):
        for j, station2 in enumerate(station_list):
            if station1 != station2:
                f_statistic = granger_test(pivot_data, station1, station2)
                if f_statistic > f_statistic_99th_percentile:
                    link_matrix[year][i, j] = f_statistic

# Create a dictionary mapping station IDs to their coordinates
station_coordinates = coordinates_data.set_index('STATION')[['EST', 'NORD']].to_dict('index')

# Get maximum distance
max_distance = 0
for i in range(len(station_list)):
    for j in range(i+1, len(station_list)):
        lon1, lat1 = station_coordinates[station_list[i]]['EST'], station_coordinates[station_list[i]]['NORD']
        lon2, lat2 = station_coordinates[station_list[j]]['EST'], station_coordinates[station_list[j]]['NORD']
        dist = haversine(lon1, lat1, lon2, lat2)
        if dist > max_distance:
            max_distance = dist

# Create and display the grids
for year in years:
    grid = np.ones((num_stations, num_stations, 3))  # Initialize grid with white background (no links)

    for i, station1 in enumerate(station_list):
        lon1, lat1 = station_coordinates[station1]['EST'], station_coordinates[station1]['NORD']
        for j, station2 in enumerate(station_list):
            if link_matrix[year][i, j] > 0:  # Check if there's a significant link
                lon2, lat2 = station_coordinates[station2]['EST'], station_coordinates[station2]['NORD']
                color = link_color(lon1, lat1, lon2, lat2, max_distance)
                grid[i, j] = mcolors.to_rgb(color)

    # Create and display the plot
    plt.figure(figsize=(10, 10))
    plt.imshow(grid, interpolation='nearest')
    plt.title(f'Autumn Adjacency Matrix for {year}')
    plt.xticks(range(num_stations), [station_order[s] for s in station_list], rotation=90)
    plt.yticks(range(num_stations), [station_order[s] for s in station_list])
    plt.grid(False)
    plt.savefig(f"4.Grid_{year}autumn.jpg", bbox_inches='tight', dpi=300)

    plt.show()
    #GR

In [ ]:
#edges spatial lenght and direction

import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt
from statsmodels.tsa.stattools import grangercausalitytests

# Haversine formula to calculate distance between two coordinates
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of Earth in kilometers
    return c * r

# Function to perform Granger causality test and return F-statistic
def granger_test(dataframe, column1, column2, max_lag=1):
    try:
        gc_test = grangercausalitytests(dataframe[[column1, column2]], maxlag=max_lag)
        f_statistic = gc_test[1][0]['ssr_ftest'][0]
        p_value = gc_test[1][0]['ssr_ftest'][1]
        return f_statistic if p_value < 0.0001 else 0
    except:
        return 0

# Load and prepare data
gauges_data = pd.read_csv('75gauges.csv')
coordinates_data = pd.read_csv('merged_COO.csv')
merged_data = pd.merge(gauges_data, coordinates_data, on='STATION')
merged_data['DATETIME'] = pd.to_datetime(merged_data['DATETIME'])
stations_of_interest = [779, 750, 683, 695, 718, 756, 706, 764, 729]
merged_data_filtered = merged_data[merged_data['STATION'].isin(stations_of_interest)]

# Define time intervals
intervals = {
    '2002-2012': range(2002, 2013),
    '2013-2023': range(2013, 2024)
}

# Initialize link_matrix
link_matrix = {}
for interval, years in intervals.items():
    link_matrix[interval] = np.zeros((len(stations_of_interest), len(stations_of_interest)))

# Calculate F-statistics and populate link_matrix
all_f_statistics = []
for interval, years in intervals.items():
    for year in years:
        #data_year = merged_data_filtered[merged_data_filtered['DATETIME'].dt.year == year]
        data_year = merged_data[(merged_data['DATETIME'].dt.year == year) & 
                            (merged_data['DATETIME'].dt.month.isin([6, 7, 8]))]
    
        pivot_data = data_year.pivot(index='DATETIME', columns='STATION', values='VALUE').fillna(0)
        
        for i, station1 in enumerate(stations_of_interest):
            for j, station2 in enumerate(stations_of_interest):
                if station1 != station2:
                    f_statistic = granger_test(pivot_data, station1, station2)
                    all_f_statistics.append(f_statistic)
                    if f_statistic > np.percentile(all_f_statistics, 75):
                        link_matrix[interval][i, j] = f_statistic

# Function to count links by distance and direction
def count_distance_direction_links(link_matrix, intervals, stations_of_interest, coordinates_data, threshold):
    results = {}
    station_coordinates = coordinates_data.set_index('STATION')[['EST', 'NORD']].to_dict('index')

    for interval, matrix in link_matrix.items():
        results[interval] = {'<50km': {'eastward': 0, 'westward': 0},
                             '50-150km': {'eastward': 0, 'westward': 0},
                             '150-250km': {'eastward': 0, 'westward': 0},
                             '>250km': {'eastward': 0, 'westward': 0}}
        
        for i, station1 in enumerate(stations_of_interest):
            for j, station2 in enumerate(stations_of_interest):
                if i != j and matrix[i, j] > threshold:
                    lon1, lat1 = station_coordinates[station1]['EST'], station_coordinates[station1]['NORD']
                    lon2, lat2 = station_coordinates[station2]['EST'], station_coordinates[station2]['NORD']
                    distance = haversine(lon1, lat1, lon2, lat2)
                    direction = 'eastward' if lon2 > lon1 else 'westward'

                    if distance < 50:
                        results[interval]['<50km'][direction] += 1
                    elif 50 <= distance < 150:
                        results[interval]['50-150km'][direction] += 1
                    elif 155 <= distance < 250:
                        results[interval]['150-250km'][direction] += 1
                    elif distance >= 250:
                        results[interval]['>250km'][direction] += 1

    return results

# Calculate and print results for each time interval
threshold = np.percentile(all_f_statistics, 75)
results = count_distance_direction_links(link_matrix, intervals, stations_of_interest, coordinates_data, threshold)
print(results)


In [ ]:
# decadal link counts
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt
from statsmodels.tsa.stattools import grangercausalitytests

# Haversine formula to calculate distance between two coordinates
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of Earth in kilometers
    return c * r

# Function to perform Granger causality test and return F-statistic
def granger_test(dataframe, column1, column2, max_lag=1):
    try:
        gc_test = grangercausalitytests(dataframe[[column1, column2]], maxlag=max_lag)
        f_statistic = gc_test[1][0]['ssr_ftest'][0]
        p_value = gc_test[1][0]['ssr_ftest'][1]
        return f_statistic if p_value < 0.0001 else 0
    except:
        return 0

# Load and prepare data
gauges_data = pd.read_csv('75gauges.csv')
coordinates_data = pd.read_csv('merged_COO.csv')
merged_data = pd.merge(gauges_data, coordinates_data, on='STATION')
merged_data['DATETIME'] = pd.to_datetime(merged_data['DATETIME'])
stations_of_interest = [779, 750, 683, 695, 718, 756, 706, 764, 729]
merged_data_filtered = merged_data[merged_data['STATION'].isin(stations_of_interest)]

# Define time intervals
intervals = {
    '2002-2012': range(2002, 2013),
    '2013-2023': range(2013, 2024)
}

# Initialize link_matrix
link_matrix = {}
for interval, years in intervals.items():
    link_matrix[interval] = np.zeros((len(stations_of_interest), len(stations_of_interest)))

# Calculate F-statistics and populate link_matrix
all_f_statistics = []
for interval, years in intervals.items():
    for year in years:
        #data_year = merged_data_filtered[merged_data_filtered['DATETIME'].dt.year == year]
        data_year = merged_data[(merged_data['DATETIME'].dt.year == year) & 
                            (merged_data['DATETIME'].dt.month.isin([9, 10, 11]))]
    
        pivot_data = data_year.pivot(index='DATETIME', columns='STATION', values='VALUE').fillna(0)
        
        for i, station1 in enumerate(stations_of_interest):
            for j, station2 in enumerate(stations_of_interest):
                if station1 != station2:
                    f_statistic = granger_test(pivot_data, station1, station2)
                    all_f_statistics.append(f_statistic)
                    if f_statistic > np.percentile(all_f_statistics, 75):
                        link_matrix[interval][i, j] = f_statistic

# Function to count links by distance and direction
def count_distance_direction_links(link_matrix, intervals, stations_of_interest, coordinates_data, threshold):
    results = {}
    station_coordinates = coordinates_data.set_index('STATION')[['EST', 'NORD']].to_dict('index')

    for interval, matrix in link_matrix.items():
        results[interval] = {
                             '50-155km': {'eastward': 0, 'westward': 0},
                             '155-350km': {'eastward': 0, 'westward': 0}
                             }
        
        for i, station1 in enumerate(stations_of_interest):
            for j, station2 in enumerate(stations_of_interest):
                if i != j and matrix[i, j] > threshold:
                    lon1, lat1 = station_coordinates[station1]['EST'], station_coordinates[station1]['NORD']
                    lon2, lat2 = station_coordinates[station2]['EST'], station_coordinates[station2]['NORD']
                    distance = haversine(lon1, lat1, lon2, lat2)
                    direction = 'eastward' if lon2 > lon1 else 'westward'

                    
                    if 50 <= distance < 155:
                        results[interval]['50-155km'][direction] += 1
                    elif 155 <= distance <= 350:
                        results[interval]['155-350km'][direction] += 1
                   
    return results

# Calculate and print results for each time interval
threshold = np.percentile(all_f_statistics, 75)
results = count_distance_direction_links(link_matrix, intervals, stations_of_interest, coordinates_data, threshold)
print(results)


In [ ]:
# link counts plot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt
from statsmodels.tsa.stattools import grangercausalitytests

# Haversine formula to calculate distance between two coordinates
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of Earth in kilometers
    return c * r

# Function to perform Granger causality test and return F-statistic
def granger_test(dataframe, column1, column2, max_lag=1):
    try:
        gc_test = grangercausalitytests(dataframe[[column1, column2]], maxlag=max_lag)
        f_statistic = gc_test[1][0]['ssr_ftest'][0]
        p_value = gc_test[1][0]['ssr_ftest'][1]
        return f_statistic if p_value < 0.0001 else 0
    except:
        return 0

# Load and prepare data
gauges_data = pd.read_csv('75gauges.csv')
coordinates_data = pd.read_csv('merged_COO.csv')
merged_data = pd.merge(gauges_data, coordinates_data, on='STATION')
merged_data['DATETIME'] = pd.to_datetime(merged_data['DATETIME'])
stations_of_interest = [779, 750, 683, 695, 718, 756, 706, 764, 729]

# Initialize link_matrix
start_year, end_year = 2002, 2023
annual_links = {year: {'50-155km': {'eastward': 0, 'westward': 0}, '155-350km': {'eastward': 0, 'westward': 0}} for year in range(start_year, end_year+1)}

all_f_statistics = []
# Analyze data year by year
for year in range(start_year, end_year+1):
    data_year = merged_data[(merged_data['DATETIME'].dt.year == year) & 
                            (merged_data['DATETIME'].dt.month.isin([9, 10, 11]))]
    pivot_data = data_year.pivot(index='DATETIME', columns='STATION', values='VALUE').fillna(0)

    for i, station1 in enumerate(stations_of_interest):
        for j, station2 in enumerate(stations_of_interest):
            if station1 != station2:
                f_statistic = granger_test(pivot_data, station1, station2)
                all_f_statistics.append(f_statistic)
                if f_statistic > np.percentile(all_f_statistics, 75):
                    lon1, lat1 = coordinates_data.loc[coordinates_data['STATION'] == station1, ['EST', 'NORD']].values[0]
                    lon2, lat2 = coordinates_data.loc[coordinates_data['STATION'] == station2, ['EST', 'NORD']].values[0]
                    distance = haversine(lon1, lat1, lon2, lat2)
                    direction = 'eastward' if lon2 > lon1 else 'westward'
                    
                    if 50 <= distance < 155:
                        annual_links[year]['50-155km'][direction] += 1
                    elif 155 <= distance <= 350:
                        annual_links[year]['155-350km'][direction] += 1

# Plotting results
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 10), sharex=True)

years = range(start_year, end_year+1)
for i, dist_category in enumerate(['50-155km', '155-350km']):
    eastward_counts = [annual_links[year][dist_category]['eastward'] for year in years]
    westward_counts = [annual_links[year][dist_category]['westward'] for year in years]

    axes[i].plot(years, eastward_counts, label='Eastward', marker='o', linestyle='-', color='red')
    axes[i].plot(years, westward_counts, label='Westward', marker='x', linestyle='--', color='blue')
    axes[i].set_title(f'Summer Annual Links 75th percentile for Distance Category {dist_category}')
    axes[i].set_ylabel('Count of Links')
    axes[i].legend()

axes[-1].set_xlabel('Year')
plt.tight_layout()
plt.savefig(f"4.links.jpg", bbox_inches='tight', dpi=300)
plt.show()


In [ ]:
# plot of strenghts
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt
from statsmodels.tsa.stattools import grangercausalitytests

# Haversine formula to calculate distance between two coordinates
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of Earth in kilometers
    return c * r

# Function to perform Granger causality test and return F-statistic
def granger_test(dataframe, column1, column2, max_lag=1):
    try:
        gc_test = grangercausalitytests(dataframe[[column1, column2]], maxlag=max_lag)
        f_statistic = gc_test[1][0]['ssr_ftest'][0]
        p_value = gc_test[1][0]['ssr_ftest'][1]
        return f_statistic if p_value < 0.0001 else 0
    except:
        return 0

# Load and prepare data
gauges_data = pd.read_csv('75gauges.csv')
coordinates_data = pd.read_csv('merged_COO.csv')
merged_data = pd.merge(gauges_data, coordinates_data, on='STATION')
merged_data['DATETIME'] = pd.to_datetime(merged_data['DATETIME'])
stations_of_interest = [779, 750, 683, 695, 718, 756, 706, 764, 729]

# Initialize link_matrix
start_year, end_year = 2002, 2023
annual_links = {year: {'50-155km': {'eastward': 0, 'westward': 0}, '155-350km': {'eastward': 0, 'westward': 0}} for year in range(start_year, end_year+1)}

all_f_statistics = []
# Analyze data year by year
for year in range(start_year, end_year+1):
    data_year = merged_data[(merged_data['DATETIME'].dt.year == year) & 
                            (merged_data['DATETIME'].dt.month.isin([9, 10, 11]))]
    pivot_data = data_year.pivot(index='DATETIME', columns='STATION', values='VALUE').fillna(0)

    for i, station1 in enumerate(stations_of_interest):
        for j, station2 in enumerate(stations_of_interest):
            if station1 != station2:
                f_statistic = granger_test(pivot_data, station1, station2)
                all_f_statistics.append(f_statistic)
                if f_statistic > np.percentile(all_f_statistics, 75):
                    lon1, lat1 = coordinates_data.loc[coordinates_data['STATION'] == station1, ['EST', 'NORD']].values[0]
                    lon2, lat2 = coordinates_data.loc[coordinates_data['STATION'] == station2, ['EST', 'NORD']].values[0]
                    distance = haversine(lon1, lat1, lon2, lat2)
                    direction = 'eastward' if lon2 > lon1 else 'westward'
                    
                    if 50 <= distance < 155:
                        annual_links[year]['50-155km'][direction] += f_statistic
                    elif 155 <= distance <= 350:
                        annual_links[year]['155-350km'][direction] += f_statistic

# Plotting results
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 10), sharex=True)

years = range(start_year, end_year+1)
for i, dist_category in enumerate(['50-155km', '155-350km']):
    eastward_sums = [annual_links[year][dist_category]['eastward'] for year in years]
    westward_sums = [annual_links[year][dist_category]['westward'] for year in years]

    axes[i].plot(years, eastward_sums, label='Eastward', marker='o', linestyle='-', color='red')
    axes[i].plot(years, westward_sums, label='Westward', marker='x', linestyle='--',color='blue')
    axes[i].set_title(f'Autumn Annual Link Strengths 75th percentile for Distance Category {dist_category}')
    axes[i].set_ylabel('Sum of F-statistics')
    axes[i].legend()

axes[-1].set_xlabel('Year')
plt.tight_layout()
plt.savefig("4.strengths.jpg", bbox_inches='tight', dpi=300)
plt.show()


In [ ]:
#NETWORK ANALYISIS Clustering Coefficient and Average Shortest Path Length Across Year
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from math import radians, cos, sin, asin, sqrt
from statsmodels.tsa.stattools import grangercausalitytests
import networkx as nx

# Haversine formula to calculate the distance between two points on the Earth
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of Earth in kilometers
    return c * r

# Function to perform Granger causality test and return F-statistic
def granger_test(dataframe, column1, column2, max_lag=1):
    try:
        gc_test = grangercausalitytests(dataframe[[column1, column2]], maxlag=max_lag)
        f_statistic = gc_test[1][0]['ssr_ftest'][0]  # F-statistic
        p_value = gc_test[1][0]['ssr_ftest'][1]  # p-value
        return f_statistic if p_value < 0.0001 else 0
    except:
        return 0  # Return 0 in case of an error

# Function to determine the color of a cell based on the link direction and distance
def link_color(lon1, lat1, lon2, lat2, max_distance):
    distance = haversine(lon1, lat1, lon2, lat2)
    normalized_distance = distance / max_distance  # Normalize distance to [0, 1]
    
    if lon2 > lon1:
        # Eastward link, use red colormap
        return mcolors.to_hex(plt.cm.Reds(normalized_distance))
    else:
        # Westward link, use blue colormap
        return mcolors.to_hex(plt.cm.Blues(normalized_distance))

# Load data
gauges_data = pd.read_csv('75gauges.csv')
coordinates_data = pd.read_csv('merged_COO.csv')

# Merge data on STATION
merged_data = pd.merge(gauges_data, coordinates_data, on='STATION')

# Convert 'DATETIME' to datetime
merged_data['DATETIME'] = pd.to_datetime(merged_data['DATETIME'])

# Define the station IDs you want to include in the network
stations_of_interest = [779, 750, 683, 695, 718, 756, 706, 764, 729]

# Filter merged_data to include only stations of interest
merged_data_filtered = merged_data[merged_data['STATION'].isin(stations_of_interest)]

# Define the station order dictionary
station_order = {
    779: "TP",
    750: "PA",
    683: "AG",
    695: "CL",
    718: "EN",
    756: "RG",
    706: "CT",
    764: "SR",
    729: "ME"
}

# Create a list of station IDs in order
station_list = list(station_order.keys())

# Process data for each year and calculate all F-statistics
all_f_statistics = []
for year in range(merged_data_filtered['DATETIME'].dt.year.min(), merged_data_filtered['DATETIME'].dt.year.max() + 1):
    data_year = merged_data_filtered[(merged_data_filtered['DATETIME'].dt.year == year)]
    pivot_data = data_year.pivot(index='DATETIME', columns='STATION', values='VALUE').fillna(0)
    for station1 in pivot_data.columns:
        for station2 in pivot_data.columns:
            if station1 != station2:
                f_statistic = granger_test(pivot_data, station1, station2)
                all_f_statistics.append(f_statistic)

# Calculate the 75th percentile of F-statistics
f_statistic_75th_percentile = np.percentile(all_f_statistics, 75)

# Initialize a matrix to store the link information for each year
years = range(merged_data_filtered['DATETIME'].dt.year.min(), merged_data_filtered['DATETIME'].dt.year.max() + 1)
num_stations = len(station_list)
link_matrix = {year: np.zeros((num_stations, num_stations)) for year in years}

# Populate the link matrix
for year in years:
    data_year = merged_data_filtered[(merged_data_filtered['DATETIME'].dt.year == year)]
    pivot_data = data_year.pivot(index='DATETIME', columns='STATION', values='VALUE').fillna(0)
    for i, station1 in enumerate(station_list):
        for j, station2 in enumerate(station_list):
            if station1 != station2:
                f_statistic = granger_test(pivot_data, station1, station2)
                if f_statistic > f_statistic_75th_percentile:
                    link_matrix[year][i, j] = f_statistic

# Create a dictionary mapping station IDs to their coordinates
station_coordinates = coordinates_data.set_index('STATION')[['EST', 'NORD']].to_dict('index')

# Get maximum distance
max_distance = 0
for i in range(len(station_list)):
    for j in range(i+1, len(station_list)):
        lon1, lat1 = station_coordinates[station_list[i]]['EST'], station_coordinates[station_list[i]]['NORD']
        lon2, lat2 = station_coordinates[station_list[j]]['EST'], station_coordinates[station_list[j]]['NORD']
        dist = haversine(lon1, lat1, lon2, lat2)
        if dist > max_distance:
            max_distance = dist

# Initialize dictionaries to store clustering coefficient and average shortest path length
clustering_coefficient = {}
avg_shortest_path_length = {}

# Calculate clustering coefficient and average shortest path length for each year
for year in years:
    G = nx.Graph()
    for station in station_list:
        lon, lat = station_coordinates[station]['EST'], station_coordinates[station]['NORD']
        G.add_node(station, pos=(lon, lat))
    for i, station1 in enumerate(station_list):
        for j, station2 in enumerate(station_list):
            if link_matrix[year][i, j] > 0:
                G.add_edge(station1, station2)
    
    if nx.is_connected(G):  # Check if the graph is connected
        clustering_coefficient[year] = nx.average_clustering(G)
        avg_shortest_path_length[year] = nx.average_shortest_path_length(G)
    else:
        # Handle the case where the graph is not connected
        clustering_coefficient[year] = 0
        avg_shortest_path_length[year] = float('inf')  # Set to infinity or any appropriate value

# Plotting code remains the same


# Plotting 75th percentile degree across years
percentile_degree_75 = {}
for year, G in network_graphs.items():
    degree_sequence = [d for n, d in G.degree()]
    percentile_degree_75[year] = np.percentile(degree_sequence, 75)

plt.figure(figsize=(10, 6))
plt.plot(list(percentile_degree_75.keys()), list(percentile_degree_75.values()), marker='o', linestyle='-')
plt.xlabel('Year')
plt.ylabel('75th Percentile Degree')
plt.title('75th Percentile Degree Across Years')
plt.grid(True)
plt.show()

# Plotting clustering coefficient and average shortest path length across years
plt.figure(figsize=(10, 6))
plt.plot(list(clustering_coefficient.keys()), list(clustering_coefficient.values()), marker='o', linestyle='-', label='Clustering Coefficient')
plt.plot(list(avg_shortest_path_length.keys()), list(avg_shortest_path_length.values()), marker='o', linestyle='-', label='Average Shortest Path Length')
plt.xlabel('Year')
plt.ylabel('Value')
plt.title('Clustering Coefficient and Average Shortest Path Length Across Years')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Plotting 75th percentile degree across years
percentile_degree_75 = {}
for year, G in network_graphs.items():
    degree_sequence = [d for n, d in G.degree()]
    percentile_degree_75[year] = np.percentile(degree_sequence, 75)

plt.figure(figsize=(10, 6))
plt.plot(list(percentile_degree_75.keys()), list(percentile_degree_75.values()), marker='o', linestyle='-')
plt.xlabel('Year')
plt.ylabel('75th Percentile Degree')
plt.title('75th Percentile Degree Across Years')
plt.xscale('log')
plt.yscale('log')
plt.grid(True)
plt.show()

In [ ]:
# f stats PDF
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from math import radians, cos, sin, asin, sqrt
from statsmodels.tsa.stattools import grangercausalitytests
from scipy.stats import gaussian_kde

# Haversine formula to calculate the distance between two points on the Earth
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of Earth in kilometers
    return c * r

# Function to perform Granger causality test and return F-statistic
def granger_test(dataframe, column1, column2, max_lag=1):
    try:
        gc_test = grangercausalitytests(dataframe[[column1, column2]], maxlag=max_lag)
        f_statistic = gc_test[1][0]['ssr_ftest'][0]  # F-statistic
        p_value = gc_test[1][0]['ssr_ftest'][1]  # p-value
        return f_statistic if p_value < 0.0001 else 0
    except:
        return 0  # Return 0 in case of an error

# Function to determine the color of a cell based on the link direction and distance
def link_color(lon1, lat1, lon2, lat2, max_distance):
    distance = haversine(lon1, lat1, lon2, lat2)
    normalized_distance = distance / max_distance  # Normalize distance to [0, 1]
    
    if lon2 > lon1:
        # Eastward link, use red colormap
        return mcolors.to_hex(plt.cm.Reds(normalized_distance))
    else:
        # Westward link, use blue colormap
        return mcolors.to_hex(plt.cm.Blues(normalized_distance))

# Load data
gauges_data = pd.read_csv('75gauges.csv')
coordinates_data = pd.read_csv('merged_COO.csv')

# Merge data on STATION
merged_data = pd.merge(gauges_data, coordinates_data, on='STATION')

# Convert 'DATETIME' to datetime
merged_data['DATETIME'] = pd.to_datetime(merged_data['DATETIME'])

# Define the station IDs you want to include in the network
stations_of_interest = [779, 750, 683, 695, 718, 756, 706, 764, 729]

# Filter merged_data to include only stations of interest
merged_data_filtered = merged_data[merged_data['STATION'].isin(stations_of_interest)]

# Define the station order dictionary
station_order = {
    779: "TP",
    750: "PA",
    683: "AG",
    695: "CL",
    718: "EN",
    756: "RG",
    706: "CT",
    764: "SR",
    729: "ME"
}

# Create a list of station IDs in order
station_list = list(station_order.keys())

# Process data for each year and calculate all F-statistics
all_f_statistics = []
for year in range(merged_data_filtered['DATETIME'].dt.year.min(), merged_data_filtered['DATETIME'].dt.year.max() + 1):
    data_year = merged_data_filtered[(merged_data_filtered['DATETIME'].dt.year == year) & 
                                     (merged_data_filtered['DATETIME'].dt.month.isin([9, 10, 11]))]
    #data_year = merged_data_filtered[(merged_data_filtered['DATETIME'].dt.year == year)]
    pivot_data = data_year.pivot(index='DATETIME', columns='STATION', values='VALUE').fillna(0)

    for station1 in pivot_data.columns:
        for station2 in pivot_data.columns:
            if station1 != station2:
                f_statistic = granger_test(pivot_data, station1, station2)
                all_f_statistics.append({'year': year, 'f_statistic': f_statistic})

# Filter F-statistics for the two time intervals
f_statistics_2002_2012 = [f_statistic for f_statistic in all_f_statistics if 2002 <= f_statistic['year'] <= 2012]
f_statistics_2013_2023 = [f_statistic for f_statistic in all_f_statistics if 2013 <= f_statistic['year'] <= 2023]

# Extract F-statistic values
f_values_2002_2012 = [f_statistic['f_statistic'] for f_statistic in f_statistics_2002_2012]
f_values_2013_2023 = [f_statistic['f_statistic'] for f_statistic in f_statistics_2013_2023]

# Calculate PDFs using Gaussian Kernel Density Estimation (KDE)
kde_2002_2012 = gaussian_kde(f_values_2002_2012)
kde_2013_2023 = gaussian_kde(f_values_2013_2023)

# Generate x values for the plot
x_values = np.linspace(0, max(max(f_values_2002_2012), max(f_values_2013_2023)), 1000)

plt.figure(figsize=(10, 6))
plt.plot(x_values, kde_2002_2012(x_values), 'o', label='2002-2012')
plt.plot(x_values, kde_2013_2023(x_values), 'o', label='2013-2023')
plt.xlabel('F-statistic')
plt.ylabel('Density')
plt.xscale('log')
plt.yscale('log')
plt.ylim(0.0001,1)
plt.title('Probability Density Function of F-statistics')
plt.legend()
plt.grid(False)
#plt.savefig(f"1.Fstat.jpg", bbox_inches='tight', dpi=300)
plt.show()


In [ ]:
#decumulative of strenght
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt
from statsmodels.tsa.stattools import grangercausalitytests
from scipy.stats import gaussian_kde
from scipy.optimize import curve_fit

# Haversine formula to calculate the distance between two points on the Earth
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of Earth in kilometers
    return c * r

# Function to perform Granger causality test and return F-statistic
def granger_test(dataframe, column1, column2, max_lag=1):
    try:
        gc_test = grangercausalitytests(dataframe[[column1, column2]], maxlag=max_lag)
        f_statistic = gc_test[1][0]['ssr_ftest'][0]  # F-statistic
        p_value = gc_test[1][0]['ssr_ftest'][1]  # p-value
        return f_statistic if p_value < 0.0001 else 0
    except:
        return 0  # Return 0 in case of an error

# Load data
gauges_data = pd.read_csv('75gauges.csv')
coordinates_data = pd.read_csv('merged_COO.csv')

# Merge data on STATION
merged_data = pd.merge(gauges_data, coordinates_data, on='STATION')

# Convert 'DATETIME' to datetime
merged_data['DATETIME'] = pd.to_datetime(merged_data['DATETIME'])

# Define the station IDs you want to include in the network
stations_of_interest = [779, 750, 683, 695, 718, 756, 706, 764, 729]

# Filter merged_data to include only stations of interest
merged_data_filtered = merged_data[merged_data['STATION'].isin(stations_of_interest)]

# Process data for each year and calculate all F-statistics
all_f_statistics = []
for year in range(merged_data_filtered['DATETIME'].dt.year.min(), merged_data_filtered['DATETIME'].dt.year.max() + 1):
    data_year = merged_data_filtered[(merged_data_filtered['DATETIME'].dt.year == year) & 
                                     (merged_data_filtered['DATETIME'].dt.month.isin([12, 1, 2]))]
    pivot_data = data_year.pivot(index='DATETIME', columns='STATION', values='VALUE').fillna(0)

    for station1 in pivot_data.columns:
        for station2 in pivot_data.columns:
            if station1 != station2:
                f_statistic = granger_test(pivot_data, station1, station2)
                all_f_statistics.append({'year': year, 'f_statistic': f_statistic})

# Filter F-statistics for the two time intervals
f_statistics_2002_2012 = [f_statistic for f_statistic in all_f_statistics if 2002 <= f_statistic['year'] <= 2012]
f_statistics_2013_2023 = [f_statistic for f_statistic in all_f_statistics if 2013 <= f_statistic['year'] <= 2023]

# Extract F-statistic values
f_values_2002_2012 = [f_statistic['f_statistic'] for f_statistic in f_statistics_2002_2012]
f_values_2013_2023 = [f_statistic['f_statistic'] for f_statistic in f_statistics_2013_2023]

# Compute ECDF for F-statistic values
def ecdf(data):
    x = np.sort(data)
    y = np.arange(1, len(data) + 1) / len(data)
    return x, y

# Compute DCDF from ECDF
def dcdf(data):
    x, y = ecdf(data)
    return x, 1 - y

# Calculate ECDF for F-statistic values for 2002-2012 and 2013-2023 intervals
ecdf_f_values_2002_2012 = ecdf(f_values_2002_2012)
ecdf_f_values_2013_2023 = ecdf(f_values_2013_2023)

# Calculate DCDF for F-statistic values for 2002-2012 and 2013-2023 intervals
dcdf_2002_2012 = dcdf(f_values_2002_2012)
dcdf_2013_2023 = dcdf(f_values_2013_2023)

# Define Q exponential distribution function with three parameters
def q_exponential(x, alpha, beta, q):
    return (1 - (1 - q) * alpha * x)**(1 / (1 - q)) * beta

# Fit Q exponential distribution to the data
def fit_q_exponential(data, x_values):
    # Filter out infinite and NaN values from the data and corresponding x_values
    valid_indices = np.isfinite(data) & np.isfinite(x_values)
    data = data[valid_indices]
    x_values = x_values[valid_indices]

    # Define initial guess for parameters
    alpha_init_guess = 0.9
    #beta_init_guess = np.max(data)
    beta_init_guess = 0.9
    q_init_guess = 1.1

    # Perform curve fitting with initial guesses
    try:
        popt, pcov = curve_fit(q_exponential, x_values, data, p0=[alpha_init_guess, beta_init_guess, q_init_guess])
        return popt
    except RuntimeError:
        # If curve fitting fails, return None
        return None

# Fit Q exponential distribution for each time interval
params_2002_2012 = fit_q_exponential(dcdf_2002_2012[1], dcdf_2002_2012[0])
params_2013_2023 = fit_q_exponential(dcdf_2013_2023[1], dcdf_2013_2023[0])

# Plot DCDFs and fitted Q exponential distributions for each time interval
plt.figure(figsize=(14, 6))

# Plot for 2002-2012 interval
plt.subplot(1, 2, 1)
plt.plot(dcdf_2002_2012[0], dcdf_2002_2012[1], 'o', label=f'DCDF (q={params_2002_2012[2]:.2f})')
if params_2002_2012 is not None:
    plt.plot(dcdf_2002_2012[0], q_exponential(dcdf_2002_2012[0], *params_2002_2012), label=f'Q Exponential Fit (q={params_2002_2012[2]:.2f})')
plt.xlabel('F-statistic')
plt.ylabel('Decumulative Probability')
plt.xscale('log')
plt.title('2002-2012 Summer')
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.grid(False)

# Plot for 2013-2023 interval
plt.subplot(1, 2, 2)
plt.plot(dcdf_2013_2023[0], dcdf_2013_2023[1], 'o', label=f'DCDF (q={params_2013_2023[2]:.2f})')
if params_2013_2023 is not None:
    plt.plot(dcdf_2013_2023[0], q_exponential(dcdf_2013_2023[0], *params_2013_2023), label=f'Q Exponential Fit (q={params_2013_2023[2]:.2f})')
plt.xlabel('F-statistic')
plt.ylabel('Decumulative Probability')
plt.xscale('log')
plt.title('2013-2023 Summer')
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.grid(False)

plt.tight_layout()
plt.savefig(f"1.Fstatdec.jpg", bbox_inches='tight', dpi=300)
plt.show()
